In this notebook we aim at computing how well we are approximating the attention scores matrix with the formula
$$
(I_nP+E)
$$

In [1]:
import numpy as np
import scipy
import pickle

Firstly we code the base functions to generate a bunch of matrices of the desired form belonging to the three sets $\Sigma_1,\Sigma_2,\Sigma_3$

In [2]:
# Diagonal matrix with error
def creatediagonal(n,coeff_sp=0.5): #d is the vector dimension of the matrix
    vect=np.ones(n)-0.1*np.random.rand(n)
    # Now we add the sparse matrix
    sparse_matrix = coeff_sp*scipy.sparse.random(n, n, density=1/n) # usually that is the density of sparse matrix.
    sparse_matrix.setdiag(0) # we do not want to affect diagonal values
    sigma_1=np.diag(vect)+sparse_matrix.todense()
    # now we normalize by rows sigma_1
    row_sums = np.array(sigma_1.sum(axis=1)).flatten()
    sigma_1=sigma_1 / row_sums[:, np.newaxis]
    return sigma_1

# Almost diagonal matrix
def context(n,w): #here w stands for window size.
    context_matrix = np.zeros((n, n))
    for i in range(n):
      for j in range(max(0, i - w), min(i + w + 1, n)):
            # Apply the dropout
          if np.random.rand() > 0.4: # dropout probability
              context_matrix[i, j] = np.random.uniform(0.7, 1)
    return context_matrix


# Syntactic matrices
def createsintact(n, w, dropout_prob=0.4,coeff_sp=0.5):
    context_matrix = np.zeros((n, n))
    for i in range(n):
      for j in range(max(0, i - w), min(i + w + 1, n)):
            # Apply the dropout
          if np.random.rand() > dropout_prob:
              context_matrix[i, j] = np.random.uniform(0.7, 1)

    # Adding noise and normalizing
    sparse_matrix = coeff_sp * scipy.sparse.random(n, n, density=1 / n)
    sparse_matrix.setdiag(0)
    sigma_2 = context_matrix + sparse_matrix.todense()
    row_sums = np.array(sigma_2.sum(axis=1)).flatten()
    sigma_2 = sigma_2 / row_sums[:, np.newaxis]

    return sigma_2

def verticalmatrix(w,coeff_v=1): # how many other words that word will attend
    matrix = np.zeros((w, w))
    matrix[:, 0] = coeff_v*np.random.uniform(size=w,low=0.7, high=1) # vertical attending.
    return matrix[:, 0]




In [3]:
def generate_positions(n, num_positions, w):
    max_position = n - w+1
    min_distance = w
    positions = []
    while len(positions) < num_positions:
        new_position = np.random.randint(1, max_position + 1)
        if all(abs(new_position - pos) >= min_distance for pos in positions):
            positions.append(new_position)

    return np.sort(positions)


def raretoken(n,num_elements,w,coeff_d=1):
    matrix = np.eye(n)
    diagonal_values = coeff_d*np.random.uniform(size=n, low=0.7, high=1)
    np.fill_diagonal(matrix, diagonal_values)
    positions=generate_positions(n, num_elements, w)
    for pos in positions:
      for i in range(w):
        matrix[pos-1+i,pos-1]=matrix[pos-1+i,pos-1+i]
        if i>0:
          matrix[pos-1+i,pos-1+i]=0

    # Adding noise and normalizing
    sparse_matrix = 0.5 * scipy.sparse.random(n, n, density=1 / n)
    sparse_matrix.setdiag(0)
    sigma_3 = matrix + sparse_matrix.todense()
    row_sums = np.array(sigma_3.sum(axis=1)).flatten()
    sigma_3 = sigma_3 / row_sums[:, np.newaxis]

    return sigma_3



In [ ]:
A=raretoken(10,2,3)

In [ ]:
print(A)

[[0.55 0.   0.   0.31 0.   0.   0.   0.   0.13 0.  ]
 [0.7  0.   0.   0.   0.3  0.   0.   0.   0.   0.  ]
 [0.65 0.   0.   0.   0.   0.35 0.   0.   0.   0.  ]
 [0.   0.   0.   0.65 0.   0.   0.   0.3  0.06 0.  ]
 [0.   0.28 0.   0.   0.72 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.71 0.29 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   1.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   1.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.   0.  ]
 [0.23 0.   0.   0.   0.   0.   0.   0.15 0.   0.61]]


Now that we have coded the matrices that approximate the attention scores according to the new formula, we compute the distance with the original matrix arising from the attention experiments. We recall that according to our definition

$$  
|H-X|:= \sum_{i,j} |h_{i,j} -x_{i,j} | \quad h_{i,j}\in H, \quad x_{i,j}\in X.
$$
It is worthy to denote that there are many hyperparameters that may be learned: $w$ the bandwidth, the dropout probability $p$, the coefficient multiplying the sparse matrix, which in our case is set to be $0.5$, the parameter num_position and the various coefficients when dealing with random generation. These parameters can have an impact on the overall performance and to obtain them several experiments must be carried out. Nevertheless, we presume that many of them may dependend on the language we are using, as it may be the case for the bandwidth $w$, clearly an unpredictable random component is also inevitable.

In [4]:
np.set_printoptions(precision=2)

In [5]:
def computedistance(A,num_pos=2,w=3,n_attempts=10):
    best_distance = float('inf')
    best_matrix = None

    # First group
    for _ in range(n_attempts):
        sigma_1 = creatediagonal(len(A))
        distance = np.sum(np.abs(A - sigma_1))
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma_1

    # Second group
    for _ in range(n_attempts):
        sigma_2 = createsintact(len(A), w)  # We fix w=3
        distance = np.sum(np.abs(A - sigma_2))
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma_2

    # Third group
    for _ in range(n_attempts):
        sigma_3 = raretoken(len(A), num_pos, w)  # These parameters can be changed: 2 rare token and window 3
        distance = np.sum(np.abs(A - sigma_3))
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma_3

    return best_matrix, best_distance

# Example
A = np.eye(10)  # Matrice di esempio
best_matrix, best_distance = computedistance(A)
print("Best matrix:")
print(best_matrix)
print("\n Minimum Distance:", best_distance)


Best matrix:
[[1.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   1.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.19 0.   0.   0.71 0.   0.   0.   0.   0.08 0.03]
 [0.   0.   0.   0.   1.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   1.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.01 0.24 0.   0.75 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.54 0.24 0.22]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.07 0.   0.93]]

 Minimum Distance: 2.1375624361020757


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
with open('/content/drive/MyDrive/Transformer/matrix_list.pkl', 'rb') as f:
    loaded_matrix_list = pickle.load(f)

In [8]:
for i in range(len(loaded_matrix_list)):
  loaded_matrix_list[i]=loaded_matrix_list[i][:16,:16] # we cut all the paddings and take the length of the sentence

In [ ]:
for i, matrix in enumerate(loaded_matrix_list):
    best_matrix, best_distance = computedistance(matrix,num_pos=2,w=4,n_attempts=50)
    print("\n Minimum Distance:", best_distance)

<ipython-input-3-00087372232b>:38: RuntimeWarning: invalid value encountered in divide
  sigma_2 = sigma_2 / row_sums[:, np.newaxis]



Distanza minima: 21.069530669032968

Distanza minima: 19.233454623228344

Distanza minima: 22.352018368252036

Distanza minima: 22.468749502906167

Distanza minima: 20.48712989798162

Distanza minima: 20.871859012299915

Distanza minima: 20.61355644223331

Distanza minima: 21.525276899413527


Let us do a grid search for the various parameters we have to check if we can reduce the distance

In [ ]:
coeff_values = np.arange(0, 1.1, 0.1)

import itertools

def grid_search_distances(A, num_pos=2, w=3, n_attempts=30, coeff_d_values=coeff_values, coeff_v_values=coeff_values, coeff_sp_values=coeff_values):
    best_distance = float('inf')
    best_matrix = None
    best_params = None
    param_combinations = list(itertools.product(coeff_d_values, coeff_v_values, coeff_sp_values))

    for coeff_d, coeff_v, coeff_sp in param_combinations:
        # Compute distance for current parameter combination
        sigma, distance = computedistance_grid(A, num_pos, w, n_attempts, coeff_d, coeff_v, coeff_sp)

        # Update best distance and matrix if necessary
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma
            best_params = (coeff_d, coeff_v, coeff_sp)

    return best_matrix, best_distance, best_params


def computedistance_grid(A, num_pos=2, w=3, n_attempts=10, coeff_d=1, coeff_v=1, coeff_sp=0.5):
    best_distance = float('inf')
    best_matrix = None

    # First group
    for _ in range(n_attempts):
        sigma_1 = creatediagonal(len(A), coeff_sp)
        distance = np.sum(np.abs(A - sigma_1))
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma_1

    # Second group
    for _ in range(n_attempts):
        sigma_2 = createsintact(len(A), w, coeff_sp=coeff_sp)  # Pass coeff_sp
        distance = np.sum(np.abs(A - sigma_2))
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma_2

    # Third group
    for _ in range(n_attempts):
        sigma_3 = raretoken(len(A), num_pos, w, coeff_d=coeff_d)  # Pass coeff_d
        distance = np.sum(np.abs(A - sigma_3))
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma_3

    return best_matrix, best_distance



In [ ]:
best_distances=[]
mean_error=[]
for i, matrix in enumerate(loaded_matrix_list):
    best_matrix, best_distance, best_params = grid_search_distances(matrix)
    print("\n Minimum Distance:", best_distance)
    best_distances.append(best_distance)
    mean_error.append(best_distance/256) # we divide by the number of elements--> 16*16
    print("Best Parameters (coeff_d, coeff_v, coeff_sp):")
    print(best_params)


In [ ]:
import pandas as pd
d = {'Head_n': np.arange(8)+1, 'Min_distance': best_distances, 'Mean Error':mean_error}
df = pd.DataFrame(data=d)
display(df)

,Head_n,Min_distance,Mean Error
0,1,17.229423,0.067302
1,2,15.748942,0.061519
2,3,21.084405,0.082361
3,4,18.796007,0.073422
4,5,16.912899,0.066066
5,6,17.232267,0.067314
6,7,18.121464,0.070787
7,8,18.711096,0.073090


### The role of $w$ and num_pos
Let us try to check what is the effect if we change the parameters $w$ and num_pos in the distance


In [ ]:
for i, matrix in enumerate(loaded_matrix_list):
    best_matrix, best_distance = computedistance(matrix,num_pos=1,w=10,n_attempts=50)
    print("\nMinimum Distance:", best_distance)


Distanza minima: 24.31196692482674

Distanza minima: 22.015880317985104

Distanza minima: 22.334913409985454

Distanza minima: 22.429522781428368

Distanza minima: 22.900087009927248

Distanza minima: 24.410364613850103

Distanza minima: 22.785620849540138

Distanza minima: 22.61149793612879


In [ ]:
coeff_values = np.arange(0, 1.1, 0.1)

import itertools

def grid_search_distances(A, num_pos=1, w=10, n_attempts=30, coeff_d_values=coeff_values, coeff_v_values=coeff_values, coeff_sp_values=coeff_values):
    best_distance = float('inf')
    best_matrix = None
    best_params = None
    param_combinations = list(itertools.product(coeff_d_values, coeff_v_values, coeff_sp_values))

    for coeff_d, coeff_v, coeff_sp in param_combinations:
        # Compute distance for current parameter combination
        sigma, distance = computedistance_grid(A, num_pos, w, n_attempts, coeff_d, coeff_v, coeff_sp)

        # Update best distance and matrix if necessary
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma
            best_params = (coeff_d, coeff_v, coeff_sp)

    return best_matrix, best_distance, best_params


def computedistance_grid(A, num_pos=1, w=10, n_attempts=10, coeff_d=1, coeff_v=1, coeff_sp=0.5):
    best_distance = float('inf')
    best_matrix = None

    # First group
    for _ in range(n_attempts):
        sigma_1 = creatediagonal(len(A), coeff_sp)
        distance = np.sum(np.abs(A - sigma_1))
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma_1

    # Second group
    for _ in range(n_attempts):
        sigma_2 = createsintact(len(A), w, coeff_sp=coeff_sp)  # Pass coeff_sp
        distance = np.sum(np.abs(A - sigma_2))
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma_2

    # Third group
    for _ in range(n_attempts):
        sigma_3 = raretoken(len(A), num_pos, w, coeff_d=coeff_d)  # Pass coeff_d
        distance = np.sum(np.abs(A - sigma_3))
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma_3

    return best_matrix, best_distance

In [ ]:
best_distances=[]
mean_error=[]
for i, matrix in enumerate(loaded_matrix_list):
    best_matrix, best_distance, best_params = grid_search_distances(matrix)
    print("\nMinimum Distance:", best_distance)
    best_distances.append(best_distance)
    mean_error.append(best_distance/256) # we divide by the number of elements--> 16*16
    print("Best Parameters (coeff_d, coeff_v, coeff_sp):")
    print(best_params)

In [ ]:
import pandas as pd
d = {'Head_n': np.arange(8)+1, 'Min_distance': best_distances, 'Mean Error':mean_error}
df = pd.DataFrame(data=d)
display(df)

,Head_n,Min_distance,Mean Error
0,1,22.584185,0.088219
1,2,20.904304,0.081657
2,3,20.630814,0.080589
3,4,19.964075,0.077985
4,5,21.000884,0.082035
5,6,22.181472,0.086646
6,7,21.396999,0.083582
7,8,21.169499,0.082693


### "Continuity" of $w$ and num_pos


In [9]:
coeff_values = np.arange(0, 1.1, 0.1)

import itertools

def grid_search_distances(A, num_pos=2, w=2, n_attempts=30, coeff_d_values=coeff_values, coeff_v_values=coeff_values, coeff_sp_values=coeff_values):
    best_distance = float('inf')
    best_matrix = None
    best_params = None
    param_combinations = list(itertools.product(coeff_d_values, coeff_v_values, coeff_sp_values))

    for coeff_d, coeff_v, coeff_sp in param_combinations:
        # Compute distance for current parameter combination
        sigma, distance = computedistance_grid(A, num_pos, w, n_attempts, coeff_d, coeff_v, coeff_sp)

        # Update best distance and matrix if necessary
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma
            best_params = (coeff_d, coeff_v, coeff_sp)

    return best_matrix, best_distance, best_params


def computedistance_grid(A, num_pos=2, w=2, n_attempts=10, coeff_d=1, coeff_v=1, coeff_sp=0.5):
    best_distance = float('inf')
    best_matrix = None

    # First group
    for _ in range(n_attempts):
        sigma_1 = creatediagonal(len(A), coeff_sp)
        distance = np.sum(np.abs(A - sigma_1))
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma_1

    # Second group
    for _ in range(n_attempts):
        sigma_2 = createsintact(len(A), w, coeff_sp=coeff_sp)  # Pass coeff_sp
        distance = np.sum(np.abs(A - sigma_2))
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma_2

    # Third group
    for _ in range(n_attempts):
        sigma_3 = raretoken(len(A), num_pos, w, coeff_d=coeff_d)  # Pass coeff_d
        distance = np.sum(np.abs(A - sigma_3))
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma_3

    return best_matrix, best_distance

In [ ]:
best_distances=[]
mean_error=[]
for i, matrix in enumerate(loaded_matrix_list):
    best_matrix, best_distance, best_params = grid_search_distances(matrix)
    print("\nMinimum Distance:", best_distance)
    best_distances.append(best_distance)
    mean_error.append(best_distance/256) # we divide by the number of elements--> 16*16
    print("Best parameters (coeff_d, coeff_v, coeff_sp):")
    print(best_params)

In [11]:
import pandas as pd
d = {'Head_n': np.arange(8)+1, 'Min_distance': best_distances, 'Mean Error':mean_error}
df = pd.DataFrame(data=d)
display(df)

,Head_n,Min_distance,Mean Error
0,1,16.087786,0.062843
1,2,16.420829,0.064144
2,3,20.548045,0.080266
3,4,19.649457,0.076756
4,5,17.699269,0.069138
5,6,16.940487,0.066174
6,7,17.220498,0.067268
7,8,17.020118,0.066485


In [12]:
# Now let's change the num_pos
coeff_values = np.arange(0, 1.1, 0.1)

import itertools

def grid_search_distances(A, num_pos=1, w=2, n_attempts=30, coeff_d_values=coeff_values, coeff_v_values=coeff_values, coeff_sp_values=coeff_values):
    best_distance = float('inf')
    best_matrix = None
    best_params = None
    param_combinations = list(itertools.product(coeff_d_values, coeff_v_values, coeff_sp_values))

    for coeff_d, coeff_v, coeff_sp in param_combinations:
        # Compute distance for current parameter combination
        sigma, distance = computedistance_grid(A, num_pos, w, n_attempts, coeff_d, coeff_v, coeff_sp)

        # Update best distance and matrix if necessary
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma
            best_params = (coeff_d, coeff_v, coeff_sp)

    return best_matrix, best_distance, best_params


def computedistance_grid(A, num_pos=1, w=2, n_attempts=10, coeff_d=1, coeff_v=1, coeff_sp=0.5):
    best_distance = float('inf')
    best_matrix = None

    # First group
    for _ in range(n_attempts):
        sigma_1 = creatediagonal(len(A), coeff_sp)
        distance = np.sum(np.abs(A - sigma_1))
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma_1

    # Second group
    for _ in range(n_attempts):
        sigma_2 = createsintact(len(A), w, coeff_sp=coeff_sp)  # Pass coeff_sp
        distance = np.sum(np.abs(A - sigma_2))
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma_2

    # Third group
    for _ in range(n_attempts):
        sigma_3 = raretoken(len(A), num_pos, w, coeff_d=coeff_d)  # Pass coeff_d
        distance = np.sum(np.abs(A - sigma_3))
        if distance < best_distance:
            best_distance = distance
            best_matrix = sigma_3

    return best_matrix, best_distance

In [ ]:
best_distances=[]
mean_error=[]
for i, matrix in enumerate(loaded_matrix_list):
    best_matrix, best_distance, best_params = grid_search_distances(matrix)
    print("\nMinimum Distance:", best_distance)
    best_distances.append(best_distance)
    mean_error.append(best_distance/256) # we divide by the number of elements--> 16*16
    print("Best parameters (coeff_d, coeff_v, coeff_sp):")
    print(best_params)

In [14]:
d = {'Head_n': np.arange(8)+1, 'Min_distance': best_distances, 'Mean Error':mean_error}
df = pd.DataFrame(data=d)
display(df)

,Head_n,Min_distance,Mean Error
0,1,16.914446,0.066072
1,2,16.056305,0.062720
2,3,20.277149,0.079208
3,4,19.856062,0.077563
4,5,17.475247,0.068263
5,6,16.390316,0.064025
6,7,17.344392,0.067752
7,8,18.341860,0.071648
